In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load the data
data_saran = pd.read_csv("/content/drive/MyDrive/Model1_Classification/CSV/data_saran2.csv")
tryout_data = pd.read_csv("/content/drive/MyDrive/Model1_Classification/CSV/MOCK_DATA.csv")

In [ ]:
tryout_data.head()

,id_user,first_name,last_name,kesalahan_analogi_verbal,kesalahan_silogisme,kesalahan_analitis,kesalahan_berhitung,kesalahan_deret_angka,kesalahan_perbandingan_kuantitatif,kesalahan_soal_cerita,...,skor_sosial_budaya,skor_tik,skor_profesionalisme,skor_anti_radikalisme,total_kesalahan_tiu,total_kesalahan_twk,total_skor_tkp,label_tiu,label_twk,label_tkp
0,1,Rina,Hidayat,1,3,2,0,0,2,4,...,4,19,35,13,16,13,127,Sedang,Tinggi,Rendah
1,2,Joko,Hidayat,2,1,3,1,2,1,0,...,16,13,24,35,14,15,148,Tinggi,Sedang,Rendah
2,3,Joko,Hidayat,0,1,1,3,0,0,2,...,10,6,22,14,17,16,58,Sedang,Sedang,Rendah
3,4,Rina,Hidayat,0,1,2,0,1,0,2,...,7,40,16,30,11,17,148,Tinggi,Sedang,Rendah
4,5,Siti,Lestari,1,1,2,0,2,0,2,...,24,22,15,28,13,17,140,Tinggi,Sedang,Rendah


In [ ]:
data_saran.head()

,kesalahan_analogi_verbal,kesalahan_silogisme,kesalahan_analitis,kesalahan_berhitung,kesalahan_deret_angka,kesalahan_perbandigan_kuantitatif,kesalahan_soal_cerita,kesalahan_ketidaksamaan,kesalahan_serial,kesalahan_analogi_figural,...,kesalahan_integritas,kesalahan_bela_negara,kesalahan_pilar_negara,kesalahan_bahasa_indonesia,skor_pelayanan_publik,skor_jejaring_kerja,skor_sosial_budaya,skor_tik,skor_profesionalisme,kesalahan_anti_radikalisme
0,1. Kosa Kata: Perluas kosa kata dengan mempela...,1. Logika Dasar: Memahami konsep dasar logika ...,1. Pemecahan Masalah Sistematis: Latih teknik ...,1. Aritmetika Dasar: Memperkuat penguasaan ope...,1. Identifikasi Pola: Pelajari berbagai pola d...,1. Konsep Rasio dan Proporsi: Memahami konsep ...,1. Pemahaman Teks: Tingkatkan kemampuan memaha...,1. Identifikasi Pola Dasar: Latih diri untuk m...,1. Analisis Urutan Gambar: Pelajari bagaimana ...,1. Pahami Relasi Antar Gambar: Latih kemampuan...,...,1. Pelajari Nilai-Nilai Etika: Pahami nilai-ni...,1. Pahami Konsep Bela Negara: Pelajari apa yan...,1. Pelajari Pancasila: Dalami lima sila Pancas...,1. Tata Bahasa: Pelajari kembali aturan penggu...,1. Prosedur: Pelajari ulang SOP (Standard Oper...,1. Membangun Jaringan: Aktiflah dalam forum-fo...,1. Pemahaman Budaya: Pelajari lebih dalam tent...,1. Penggunaan Teknologi: Ikuti kursus online a...,1. Etika Kerja: Pelajari dan pahami kode etik ...,1. Pemahaman: Pelajari materi tentang radikali...


In [ ]:
# Define score categorization function
def categorize_score(score, min_passing_score):
    if score < min_passing_score:
        return "Rendah"
    elif score < min_passing_score + 20:
        return "Sedang"
    else:
        return "Tinggi"

# Define minimum passing scores
min_tiu_score = 80
min_twk_score = 65
min_tkp_score = 166

# Calculate total scores for TIU, TWK, and TKP
tryout_data['total_tiu_score'] = 175 - (tryout_data[['kesalahan_analogi_verbal', 'kesalahan_silogisme', 'kesalahan_analitis', 'kesalahan_berhitung', 'kesalahan_deret_angka', 'kesalahan_perbandingan_kuantitatif', 'kesalahan_soal_cerita', 'kesalahan_ketidaksamaan', 'kesalahan_serial', 'kesalahan_analogi_figural']].sum(axis=1) * 5)
tryout_data['total_twk_score'] = 150 - (tryout_data[['kesalahan_nasionalisme', 'kesalahan_integritas', 'kesalahan_bela_negara', 'kesalahan_pilar_negara', 'kesalahan_bahasa_indonesia']].sum(axis=1) * 5)
tryout_data['total_tkp_score'] = tryout_data[['skor_pelayanan_publik', 'skor_jejaring_kerja', 'skor_sosial_budaya', 'skor_tik', 'skor_profesionalisme', 'skor_anti_radikalisme']].sum(axis=1)

# Categorize scores into labels
tryout_data['label_tiu'] = tryout_data['total_tiu_score'].apply(lambda x: categorize_score(x, min_tiu_score))
tryout_data['label_twk'] = tryout_data['total_twk_score'].apply(lambda x: categorize_score(x, min_twk_score))
tryout_data['label_tkp'] = tryout_data['total_tkp_score'].apply(lambda x: categorize_score(x, min_tkp_score))

# Prepare training data
X_tiu = tryout_data[['kesalahan_analogi_verbal', 'kesalahan_silogisme', 'kesalahan_analitis', 'kesalahan_berhitung', 'kesalahan_deret_angka', 'kesalahan_perbandingan_kuantitatif', 'kesalahan_soal_cerita', 'kesalahan_ketidaksamaan', 'kesalahan_serial', 'kesalahan_analogi_figural']]
y_tiu = tryout_data['label_tiu']
X_twk = tryout_data[['kesalahan_nasionalisme', 'kesalahan_integritas', 'kesalahan_bela_negara', 'kesalahan_pilar_negara', 'kesalahan_bahasa_indonesia']]
y_twk = tryout_data['label_twk']
X_tkp = tryout_data[['skor_pelayanan_publik', 'skor_jejaring_kerja', 'skor_sosial_budaya', 'skor_tik', 'skor_profesionalisme', 'skor_anti_radikalisme']]
y_tkp = tryout_data['label_tkp']

In [ ]:
# Encode labels
label_encoder_tiu = LabelEncoder()
label_encoder_twk = LabelEncoder()
label_encoder_tkp = LabelEncoder()

y_tiu = label_encoder_tiu.fit_transform(y_tiu)
y_twk = label_encoder_twk.fit_transform(y_twk)
y_tkp = label_encoder_tkp.fit_transform(y_tkp)

# Split data
X_train_tiu, X_test_tiu, y_train_tiu, y_test_tiu = train_test_split(X_tiu, y_tiu, test_size=0.2, random_state=42)
X_train_twk, X_test_twk, y_train_twk, y_test_twk = train_test_split(X_twk, y_twk, test_size=0.2, random_state=42)
X_train_tkp, X_test_tkp, y_train_tkp, y_test_tkp = train_test_split(X_tkp, y_tkp, test_size=0.2, random_state=42)

# Define model creation function
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create models
model_tiu3 = create_model(X_train_tiu.shape[1])
model_twk3 = create_model(X_train_twk.shape[1])
model_tkp3 = create_model(X_train_tkp.shape[1])

# Custom Early Stopping Callback
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_accuracy', value=0.98):
        super(CustomEarlyStopping, self).__init__()
        self.monitor = monitor
        self.value = value

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is not None:
            if current >= self.value:
                print(f"\nStopping training as {self.monitor} reached {self.value}")
                self.model.stop_training = True

# Initialize early stopping callback
custom_early_stopping = CustomEarlyStopping(monitor='val_accuracy', value=0.98)
callbacks = [custom_early_stopping]

# Train models
model_tiu3.fit(X_train_tiu, y_train_tiu, epochs=30, validation_split=0.3, callbacks=callbacks)
model_twk3.fit(X_train_twk, y_train_twk, epochs=30, validation_split=0.3, callbacks=callbacks)
model_tkp3.fit(X_train_tkp, y_train_tkp, epochs=30, validation_split=0.3, callbacks=callbacks)

Epoch 1/30
18/18 [==============================] - 1s 20ms/step - loss: 1.5979 - accuracy: 0.3929 - val_loss: 1.1688 - val_accuracy: 0.3375
Epoch 2/30
18/18 [==============================] - 0s 10ms/step - loss: 1.3592 - accuracy: 0.4482 - val_loss: 1.2569 - val_accuracy: 0.3292
Epoch 3/30
18/18 [==============================] - 0s 9ms/step - loss: 1.2189 - accuracy: 0.5054 - val_loss: 1.3470 - val_accuracy: 0.3042
Epoch 4/30
18/18 [==============================] - 0s 9ms/step - loss: 1.0454 - accuracy: 0.5232 - val_loss: 1.3840 - val_accuracy: 0.3042
Epoch 5/30
18/18 [==============================] - 0s 11ms/step - loss: 0.9599 - accuracy: 0.5821 - val_loss: 1.3992 - val_accuracy: 0.3042
Epoch 6/30
18/18 [==============================] - 0s 12ms/step - loss: 0.8958 - accuracy: 0.6089 - val_loss: 1.3717 - val_accuracy: 0.3125
Epoch 7/30
18/18 [==============================] - 0s 8ms/step - loss: 0.9068 - accuracy: 0.6018 - val_loss: 1.3365 - val_accuracy: 0.3167
Epoch 8/30
18/18

In [ ]:
# Evaluate models
test_loss_tiu3, test_accuracy_tiu3 = model_tiu3.evaluate(X_test_tiu, y_test_tiu)
test_loss_twk3, test_accuracy_twk3 = model_twk3.evaluate(X_test_twk, y_test_twk)
test_loss_tkp3, test_accuracy_tkp3 = model_tkp3.evaluate(X_test_tkp, y_test_tkp)

print(f"TIU3 Test Loss: {test_loss_tiu3}, Test Accuracy: {test_accuracy_tiu3}")
print(f"TWK3 Test Loss: {test_loss_twk3}, Test Accuracy: {test_accuracy_twk3}")
print(f"TKP3 Test Loss: {test_loss_tkp3}, Test Accuracy: {test_accuracy_tkp3}")

7/7 [==============================] - 0s 3ms/step - loss: 0.5384 - accuracy: 0.9650
TIU3 Test Loss: 0.35548073053359985, Test Accuracy: 0.8299999833106995
TWK3 Test Loss: 0.2803269326686859, Test Accuracy: 0.9049999713897705
TKP3 Test Loss: 0.5383786559104919, Test Accuracy: 0.9649999737739563


In [ ]:
def predict_and_feedback(user_data):
    # # Predict for TIU
    tiu_predictions = model_tiu3.predict(user_data[['kesalahan_analogi_verbal', 'kesalahan_silogisme', 'kesalahan_analitis', 'kesalahan_berhitung', 'kesalahan_deret_angka', 'kesalahan_perbandingan_kuantitatif', 'kesalahan_soal_cerita', 'kesalahan_ketidaksamaan', 'kesalahan_serial', 'kesalahan_analogi_figural']])
    tiu_predictions_labels = label_encoder_tiu.inverse_transform(np.argmax(tiu_predictions, axis=1))

    # Predict for TWK
    twk_predictions = model_twk3.predict(user_data[['kesalahan_nasionalisme', 'kesalahan_integritas', 'kesalahan_bela_negara', 'kesalahan_pilar_negara', 'kesalahan_bahasa_indonesia']])
    twk_predictions_labels = label_encoder_twk.inverse_transform(np.argmax(twk_predictions, axis=1))

    # Predict for TKP
    tkp_predictions = model_tkp3.predict(user_data[['skor_pelayanan_publik', 'skor_jejaring_kerja', 'skor_sosial_budaya', 'skor_tik', 'skor_profesionalisme', 'skor_anti_radikalisme']])
    tkp_predictions_labels = label_encoder_tkp.inverse_transform(np.argmax(tkp_predictions, axis=1))

    user_data['total_tiu_score'] = 175 - (user_data[['kesalahan_analogi_verbal', 'kesalahan_silogisme', 'kesalahan_analitis', 'kesalahan_berhitung', 'kesalahan_deret_angka', 'kesalahan_perbandingan_kuantitatif', 'kesalahan_soal_cerita', 'kesalahan_ketidaksamaan', 'kesalahan_serial', 'kesalahan_analogi_figural']].sum(axis=1) * 5)
    user_data['total_twk_score'] = 150 - (user_data[['kesalahan_nasionalisme', 'kesalahan_integritas', 'kesalahan_bela_negara', 'kesalahan_pilar_negara', 'kesalahan_bahasa_indonesia']].sum(axis=1) * 5)
    user_data['total_tkp_score'] = user_data[['skor_pelayanan_publik', 'skor_jejaring_kerja', 'skor_sosial_budaya', 'skor_tik', 'skor_profesionalisme', 'skor_anti_radikalisme']].sum(axis=1)

    tiu_errors = user_data[['kesalahan_analogi_verbal', 'kesalahan_silogisme', 'kesalahan_analitis', 'kesalahan_berhitung', 'kesalahan_deret_angka', 'kesalahan_perbandingan_kuantitatif', 'kesalahan_soal_cerita', 'kesalahan_ketidaksamaan', 'kesalahan_serial', 'kesalahan_analogi_figural']]
    twk_errors = user_data[['kesalahan_nasionalisme', 'kesalahan_integritas', 'kesalahan_bela_negara', 'kesalahan_pilar_negara', 'kesalahan_bahasa_indonesia']]
    tkp_scores = user_data[['skor_pelayanan_publik', 'skor_jejaring_kerja', 'skor_sosial_budaya', 'skor_tik', 'skor_profesionalisme', 'skor_anti_radikalisme']]

    tiu_feedback_col = tiu_errors.columns[np.argmax(tiu_errors.values)]
    twk_feedback_col = twk_errors.columns[np.argmax(twk_errors.values)]
    tkp_feedback_col = tkp_scores.columns[np.argmin(tkp_scores.values)]
    tkp_best_col = tkp_scores.columns[np.argmax(tkp_scores.values)]

    tiu_feedback = data_saran[tiu_feedback_col].values[0]
    twk_feedback = data_saran[twk_feedback_col].values[0]
    tkp_feedback = data_saran[tkp_feedback_col].values[0]

    tiu_feedback_col_clean = tiu_feedback_col.replace("kesalahan_", "").replace("_", " ").title()
    twk_feedback_col_clean = twk_feedback_col.replace("kesalahan_", "").replace("_", " ").title()
    tkp_feedback_col_clean = tkp_feedback_col.replace("skor_", "").replace("_", " ").title()
    tkp_best_col_clean = tkp_best_col.replace("skor_", "").replace("_", " ").title()

    return (f"Saat ini skor untuk TIU Anda adalah {tiu_predictions_labels[0]}  dengan nilai {user_data['total_tiu_score'].values[0]} dari 175\n"
            f"Di bagian TIU Anda harus meningkatkan materi {tiu_feedback_col_clean}\nBerikut ini saran untuk meningkatkan nilai Anda: \n{tiu_feedback}\n\n"
            f"Saat ini nilai TWK Anda adalah {twk_predictions_labels[0]} dengan nilai {user_data['total_twk_score'].values[0]} dari 150\n"
            f"Di bagian TWK Anda harus meningkatkan materi {twk_feedback_col_clean}\nBerikut ini saran untuk meningkatkan nilai Anda: \n{twk_feedback}\n\n"
            f"Saat ini nilai TKP Anda adalah {tkp_predictions_labels[0]} dengan nilai {user_data['total_tkp_score'].values[0]} dari 225.\n"
            f"Di bagian TKP Anda harus meningkatkan materi {tkp_feedback_col_clean}\nBerikut ini saran untuk meningkatkan nilai Anda: \n{tkp_feedback}")

In [ ]:
print(data_saran.columns)

Index(['kesalahan_analogi_verbal', 'kesalahan_silogisme', 'kesalahan_analitis',
       'kesalahan_berhitung', 'kesalahan_deret_angka',
       'kesalahan_perbandigan_kuantitatif', 'kesalahan_soal_cerita',
       'kesalahan_ketidaksamaan', 'kesalahan_serial',
       'kesalahan_analogi_figural', 'kesalahan_nasionalisme',
       'kesalahan_integritas', 'kesalahan_bela_negara',
       'kesalahan_pilar_negara', 'kesalahan_bahasa_indonesia',
       'skor_pelayanan_publik', 'skor_jejaring_kerja', 'skor_sosial_budaya',
       'skor_tik', 'skor_profesionalisme', 'kesalahan_anti_radikalisme'],
      dtype='object')


In [ ]:
user_data_dict = {
    'kesalahan_analogi_verbal': [5],
    'kesalahan_silogisme': [5],
    'kesalahan_analitis': [5],
    'kesalahan_berhitung': [5],
    'kesalahan_deret_angka': [5],
    'kesalahan_perbandingan_kuantitatif': [5],
    'kesalahan_soal_cerita': [5],
    'kesalahan_ketidaksamaan': [5],
    'kesalahan_serial': [0],
    'kesalahan_analogi_figural': [0],
    'kesalahan_nasionalisme': [0],
    'kesalahan_integritas': [0],
    'kesalahan_bela_negara': [0],
    'kesalahan_pilar_negara': [0],
    'kesalahan_bahasa_indonesia': [0],
    'skor_pelayanan_publik': [1],
    'skor_jejaring_kerja': [1],
    'skor_sosial_budaya': [1],
    'skor_tik': [1],
    'skor_profesionalisme': [1],
    'skor_anti_radikalisme': [1]
}

user_data = pd.DataFrame(user_data_dict)

# Predict and get feedback
feedback = predict_and_feedback(user_data)
print(feedback)

1/1 [==============================] - 0s 35ms/step
Saat ini skor untuk TIU Anda adalah Rendah  dengan nilai -25 dari 175
Di bagian TIU Anda harus meningkatkan materi Analogi Verbal
Berikut ini saran untuk meningkatkan nilai Anda: 
1. Kosa Kata: Perluas kosa kata dengan mempelajari sinonim dan antonim.
2. Hubungan Kata: Pelajari berbagai jenis hubungan antara kata, seperti sebab-akibat, bagian-keseluruhan, dan fungsi.
3. Tes Verbal: Latihan soal-soal analogi verbal dari buku-buku atau sumber online.
4. Pembacaan Luas: Membaca berbagai jenis teks untuk memahami konteks penggunaan kata yang berbeda.

Saat ini nilai TWK Anda adalah Tinggi dengan nilai 150 dari 150
Di bagian TWK Anda harus meningkatkan materi Nasionalisme
Berikut ini saran untuk meningkatkan nilai Anda: 
1. Pelajari Sejarah Indonesia: Tingkatkan pemahaman tentang sejarah perjuangan kemerdekaan Indonesia, pahlawan nasional, dan peristiwa penting yang membentuk nasionalisme.
2. Kenali Simbol Nasional: Pelajari arti dan sejar

In [ ]:
# model_twk.save("model_twk.h5")
# reconstructed_model = tf.keras.models.load_model("model_twk.h5")
# tf.saved_model.save(model_twk)

# model_twk.save("model_tiu.h5")
# reconstructed_model = tf.keras.models.load_model("model_tiu.h5")
# tf.saved_model.save(model_tiu)

model_tiu3.save("model_tiu3.h5")
reconstructed_model = tf.keras.models.load_model("model_tiu3.h5")
tf.saved_model.save(model_tiu3)

TypeError: save() missing 1 required positional argument: 'export_dir'